In [107]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

import os

In [108]:
%%time
df = pd.read_pickle('../data/processed/EURUSD-15m-20210101-20241231_FEATURES.pkl')

CPU times: user 1.93 ms, sys: 23 ms, total: 25 ms
Wall time: 24 ms


In [109]:
df['ret'] = np.log(df['close']).diff().shift(-1)

In [110]:
r_threshold = df['ret'].abs().quantile(.3)

In [111]:
r_threshold

np.float64(0.00010253511648177712)

In [112]:
df['bin'] = np.zeros_like(df['close'])
df.loc[df['ret'] > r_threshold, 'bin'] = 1
df.loc[df['ret'] < -r_threshold, 'bin'] = -1

In [113]:
df['bin'].value_counts()

bin
 1.0    34739
-1.0    34417
 0.0    29640
Name: count, dtype: int64

## Save events to disk

In [335]:
labeled_events.to_pickle(DIRECTION_LABEL_FILE_PATH)

In [336]:
DIRECTION_LABEL_FILE_PATH

PosixPath('../data/labels/direction_labels/USDJPY-5m-20210101-20241231-CUSUM_0.001-TB.pkl')